# Pytorch Template

In [ ]:
# pytorch
import torch
from torch import nn

# torchinfo
%pip install torchinfo
from torchinfo import summary

# torchvision (For Picture and Video Tasks)
import torchvision

# standard data handling
import pandas as pd
import numpy as np

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

# machine learning
import sklearn
from sklearn.model_selection import train_test_split

# system
from pathlib import Path
from timeit import default_timer as timer
import requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Check the available pytorch and Cuda (GPU) Version

In [ ]:
# pytroch and cuda version
print(f" Pytorch and cuda version: {torch.__version__}")

# trochvision and cuda version
print(f" Torchvision and cuda version: {torchvision.__version__}")

 Pytorch and cuda version: 2.0.1+cu118
 Torchvision and cuda version: 0.15.2+cu118


### Check the available device

1. CPU (Default)
2. Cuda (GPU acceleration is accessible)

In [ ]:
# make device agnostic code (default is cpu)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Available device is: {device}")

Available device is: cuda


### Import Helper files

In [ ]:
# filenames
filenames = {"pytorch_helper_functions.py": "https://raw.githubusercontent.com/sl2000stat/PytorchIntroduction/master/pytorch_helper_functions.py",
             "training.py": "https://raw.githubusercontent.com/sl2000stat/PytorchIntroduction/master/training.py",
             "make_predictions.py":"https://raw.githubusercontent.com/sl2000stat/PytorchIntroduction/master/make_predictions.py",
             "validation.py":"https://raw.githubusercontent.com/sl2000stat/PytorchIntroduction/master/validation.py"}

for filename, file_path in filenames.items():

  # download helper functions from repo
  if Path(filename).is_file():
    print(f"{filename} already exists. Skipping download")

  else:

    request = requests.get(file_path)
    with open(filename, "wb") as f:
      f.write(request.content)

    print(f"Downloaded {filename}.")

pytorch_helper_functions.py already exists. Skipping download
training.py already exists. Skipping download


In [ ]:
# import functions from your own python scripts
from pytorch_helper_functions import set_global_seed, print_train_time, save_model, load_model
from training import training
from make_predictions import make_predictions

In [ ]:
# set the global seed
set_global_seed(42)

### General Pytorch Workflow

![](https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/images/01_a_pytorch_workflow.png)

1. **Getting data ready:** Data can be almost anything but to get started we're going to create a simple straight line
2. **Building a model:**	Here we'll create a model to learn patterns in the data, we'll also choose a loss function, optimizer and build a training loop.
3. **Fitting the model to data (training):** We've got data and a model, now let's let the model (try to) find patterns in the (training) data.
4. **Making predictions and evaluating a model (inference):**	Our model's found patterns in the data, let's compare its findings to the actual (testing) data.
5. **Tune the model:**	Fine tune the hyperparameter and select the optimal model.
6. **Saving and loading a model:**	You may want to use your model elsewhere, or come back to it later, here we'll cover that.


### 1. Get the Data

1. Download the Data
2. Explore the Data


### 1.1 Download the data

Dowload the zip data and extract it into a google colab folder

In [ ]:
# setup path to data folder
data_path = Path("data/")
image_path = data_path / "filename"

# If the image folder doesn't exist, download it and prepare it
if image_path.is_dir():
    print(f"{image_path} directory already exists.")
else:
    print(f"Did not find {image_path} directory, creating one.")
    image_path.mkdir(parents=True, exist_ok=True)

    # Download pizza, steak, sushi data
    with open(data_path / "filename.zip", "wb") as f:
        request = requests.get("githubrawfile")
        print("Downloading the data.")
        f.write(request.content)

    # Unzip pizza, steak, sushi data
    with zipfile.ZipFile(data_path / "filename.zip", "r") as zip_ref:
        print("Unzipping the data.")
        zip_ref.extractall(image_path)

### 1.2 Explore the Data

Data Visualization, Preperation and Exploration

In [ ]:
# How many samples are there?
print(f"Traning Samples {len(train_data)} | Validation Samples {len(val_data)}")

In [ ]:
# classes
class_names = train_data.classes
print(f"There are {len(class_names)} classes | {len(class_names)}")

### 1.6 Create Mini Batches

The DataLoader turns our data into a python iterable and allows us to divide the data into mini batches. (https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)

In [ ]:
# batch size hyperparameter
BATCH_SIZE = 32

# construct mini batches for the train and test data
train_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=True)

# check the dimensions
print(f"Length of the train DataLoader: {len(train_dataloader)} batches of {BATCH_SIZE}. (Orignially {len(training_data)})")
print(f"Length of the test DataLoader: {len(val_dataloader)} batches of {BATCH_SIZE}.  (Orignially {len(val_data)})")

### 2. Build and train your model

1. Build your own model or use an existing architecture
1. Pick a loss function and optimizer
3. Train the model and compute the train and validation scores

### 2.1 Build your own model or use an existing architecture

1. Build the model
2. Create a model instance
3. Check the Dimensions

In [ ]:
# build the model (start with a baseline model and increase the complexity or use an exisiting model architecture)
class PytorchModel(nn.Module):
  """This is the Pytorch Model class. Since it inherits from nn.Module we have to override the forward() method"""

  def __init__(self, input_shape:int,hidden_units:int,output_shape:int):
    """Constructor Initialization. Calls the super constructor and initializes our model."""

    # call the super constructor
    super().__init__()

    # create the model in several blocks

    self.block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,out_channels=hidden_units, kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,out_channels=hidden_units, kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
    )


    self.block_2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units,out_channels=hidden_units, kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,out_channels=hidden_units, kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2),
    )

    self.fully_connceted_layer = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=490, out_features=output_shape),
    )

  def forward(self,X):
    """
    This function is mandatory in each pytorch model and calculates the foward pass.

    :param X: tensor: The X data
    :return y: tensor: The y data (prediction)
    """

    # perform the calculations
    X = self.block_1(X)
    # print(f"Dimension of model output: {X.shape}")
    X = self.block_2(X)
    # print(f"Dimension of model output: {X.shape}")
    X = self.fully_connceted_layer(X)
    # print(f"Dimension of model output: {X.shape}")

    return X

In [ ]:
# Create a model instance
model = ""

# send the model to the right device
model = model.to(device)

# print the model
summary(model(),input_size=[])

AttributeError: ignored

In [ ]:
# create a dummy tensor with the same dimensions as your data, add batch dimensiona nd send it to your device
dummy_tensor = torch.randn(size=()).unsqueeze(0).to(device)

# pass the data through your model
X_dummy = model(dummy_tensor)

### 2.2 Pick a loss function & optimizer

1. Available loss functions: https://pytorch.org/docs/stable/nn.html#loss-functions
2. Available optimizer: https://pytorch.org/docs/stable/optim.html#algorithms



In [ ]:
# set up a loss function
loss_function = ""

# learning rate
LEARNING_RATE = ""

# setup an optimizer
optimizer = ""

### 2.3 Trainging and Validation Loop

In [ ]:
# number of epochs
EPOCHS = 3

# timing
train_time_start = timer()

# train and valudation loop
df_scores = training(EPOCHs=EPOCHS, model="", train_dataloader="",
             val_dataloader="", loss_function=loss_function
             optimizer=optimizer, device=device)

# calculate the training time
train_time_end = timer()
total_train_time = print_train_time(train_time_start, train_time_end, device=str(next(model.parameters()).device))

### 3. Model Evaluation

Depending on the loss function and model task there exist different evaluation methods

1. Regression
2. Classification
3. Image/Video Classification/Detection
4. Text Data

### 3.2 Classification

Evaluate the Classification.

1. Plot the Loss Curves
2. Confusion Matrix
3. Roc/Auc Scores

In [ ]:
# whether to run this cell or not
classification_task = True

if classification_task:
  pass

### 4. Making Final predictions

1. Get and transform the Data
2. Make the predictions
3. Convert the numerical predictions to human readable outputs

In [ ]:
# get and transform the data

In [ ]:
# make the predictions
predictions = make_predictions(X="", model="", device=device)

In [ ]:
# convert predictions (numerical) to prediction labels

### 5. Saving and loading a model

In [ ]:
# create directory models if it doesn't exist
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exit_ok = True)

# name the model
MODEL_NAME = "model.pth"

# create the model path
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

### 5.1 Save the model


In [ ]:
# save the sate dict of your trained model(recommended)
save_model(model, MODEL_SAVE_PATH)

### 5.2 Load the model

If you save just the parameters of the model and not the entire model, we have to create a new instance of your model class and load the saved stat_dict() into the new model

In [ ]:
# create an instance of the new model
model = ""

# load the saved sate dict in the model
model = load_model(model, MODEL_SAVE_PATH)